In [1]:
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
from scipy.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from math import sqrt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

2024-03-27 12:09:43.024889: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
import numpy as np

In [3]:
def json_to_prices(json):
    df = pd.read_json(json)
    prices = df['prices']
    coin_prices = pd.DataFrame(prices.tolist(), columns=['time', 'price'])
    prices = coin_prices['price']
    return prices

In [4]:
daily_prices = json_to_prices('data/dailyethereum.json')

In [5]:
print(daily_prices)

0      1830.306651
1      1913.816249
2      1893.714274
3      1901.078310
4      1874.694885
          ...     
271    2974.131586
272    2926.285913
273    2988.317384
274    3113.408298
275    3173.629947
Name: price, Length: 276, dtype: float64


## Linear Extrapolation

In [7]:
daily_prices_list = daily_prices.tolist()
day_indices = np.arange(len(daily_prices_list))
hour_indices = np.linspace(0, len(daily_prices_list) - 1, len(daily_prices_list) * 24)
hourly_prices = np.interp(hour_indices, day_indices, daily_prices_list)

In [8]:
print(hourly_prices)

[1830.30665132 1833.77413425 1837.24161718 ... 3168.628904   3171.12942542
 3173.62994684]


## Extrapolation using ARIMA

In [10]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

In [25]:
hourly_prices = []

for i in range(5, len(daily_prices_list)-1):
    # Fit an ARIMA model to the current day's prices
    model = ARIMA(daily_prices_list[:i+2], order=(2,1,0))
    model_fit = model.fit()

    # Forecast the next day's price
    forecast = model_fit.forecast(steps=1)

    # Interpolate the hourly prices for the current day
    current_day_prices = np.linspace(daily_prices_list[i], forecast[0], 24)

    # Add the hourly prices to the list
    hourly_prices.extend(current_day_prices)

    # Convert the list to a numpy array
hourly_prices = np.array(hourly_prices)

In [27]:
print(hourly_prices)
print(len(hourly_prices))

[1861.72538695 1862.3068235  1862.88826006 ... 3166.29657142 3168.81506062
 3171.33354982]
6480
